In [1]:
import numpy as np

In [50]:
def read_POSCAR(filename):
    
    f = open(filename, "r")
    lines = f.readlines()
    f.close()
    
    scale = float(lines[1])
    alat = []
    for i in range(3):
        a = [float(val) for val in lines[i+2].split()]
        alat.append(a)
    alat = np.array(alat)  
    alat = alat*scale
    
    symbols = lines[5].split()
    count = [int(val) for val in lines[6].split()]
    
    direct_cartesian = lines[7].split()[0]
    direct_cartesian = direct_cartesian.lower()
    
    coordinates = []
    atoms = []
    line_count = 8
    for s in range(len(symbols)):
        symbol = symbols[s]
        c = count[s]
        for i in range(c):
            coord = [float(val) for val in lines[line_count].split()]
            line_count = line_count + 1
            coordinates.append(coord)
            atoms.append(symbol)
            
    # handle direct/cartesian coordinates
    if direct_cartesian[0]  == "c":
        cart = np.array(coordinates)
        inv_alat = np.linalg.inv(alat)
        direct = np.matmul(cart, inv_alat)
    else:
        direct = np.array(coordinates)
        cart = np.matmul(direct, alat)
    
    structure = {'cart' : cart, 'direct' : direct, 'alat' : alat, 'atoms' : atoms}
    return structure

In [74]:
structure = read_POSCAR('POSCAR')
for i in range(len(data['atoms'])):
    print(data['atoms'][i],  data['direct'][i], data['cart'][i])

Si [0. 0. 0.] [0. 0. 0.]
Si [0.5 0.5 0. ] [2.715 2.715 0.   ]
Si [0.5 0.  0.5] [2.715 0.    2.715]
Si [0.  0.5 0.5] [0.    2.715 2.715]
Si [0.25 0.25 0.25] [1.3575 1.3575 1.3575]
Si [0.75 0.75 0.25] [4.0725 4.0725 1.3575]
Si [0.75 0.25 0.75] [4.0725 1.3575 4.0725]
Si [0.25 0.75 0.75] [1.3575 4.0725 4.0725]


In [81]:
def write_xyz(filename, data):
    
    f = open(filename, "w")
    
    natom = len(data['atoms'])
    f.write("%d \n" % (natom))
    
    alat = data['alat']
    a1 = [str(val) for val in alat[0]]
    a1 = " ".join(a1)
    a2 = [str(val) for val in alat[1]]
    a2 = " ".join(a2)
    a3 = [str(val) for val in alat[2]]
    a3 = " ".join(a3)
    f.write('Lattice= "%s %s %s"\n' % (a1, a2, a3))

    atoms = data['atoms']
    cart = data['cart']
    for i in range(natom):
        f.write("%s %15.3f %10.12f %5.3f\n" % (atoms[i], cart[i][0], cart[i][1], cart[i][2]))
    
    f.close()

In [82]:
write_xyz('test.xyz', structure)